# Bank Marketing Classifier

### Introdução ao tema

O marketing está presente nas nossas vidas muito mais do que imaginamos. Faça uma caminhada pelas ruas da cidade, uma busca no seu navegador, ligue a televisão ou o rádio, abra sua rede social e você será impactado por alguma ação de marketing.

Mas o que é marketing?
Para Philip Kotler, um dos teóricos mais renomados da área, define marketing como:
> *Marketing é a ciência e arte de explorar, criar e proporcionar valor para satisfazer necessidades de um público-alvo com rendibilidade.*

O campo do marketing é vasto e inclui não apenas o ato de vender um produto ou serviço, mas tudo relacionado ao planejamento, pesquisa e posicionamento de mercado. Em outras palavras, pode-se dizer que o marketing é como uma balança entre o que os clientes desejam e os objetivos da empresa. Afinal, um bom marketing precisa criar valor para ambas as partes: para a empresa e para o consumidor.

Vale ressaltar que marketing é uma palavra em inglês, derivada de market (mercado). Portanto, marketing não é apenas vender produtos ou serviços, engloba também outras atividades relacionadas ao mercado.

**Bank Marketing**<br/>

Um tipo de instituição que aplica marketing no seu dia a dia são os bancos, para isso, damos o nome Bank Marketing (Marketing Bancário). O marketing bancário é a prática de atrair e adquirir novos clientes por meio de estratégias de mídia tradicional e mídia digital. O uso dessas estratégias de mídia ajuda a determinar que tipo de cliente é atraído por uma determinada instituição. Isso também inclui diferentes instituições bancárias que usam propositalmente diferentes estratégias para atrair o tipo de cliente com o qual desejam fazer negócios.

E você sabe como as principais empresas fazem atualmente para aplicar estratégias de Marketing?<br/>
Se você respondeu, "elas aplicam técnicas de Inteligência Artificial e Machine Learning para entender e avaliar o comportamento dos seus clientes". Parabéns, você acertou!
Mas antes de explicar com elas fazem isso, vamos começar entendendo um pouco sobre o que é Machine Learning.

**Machine Learning**<br/>
O Machine Learning, ou aprendizado de máquina. É um subcampo da inteligência artificial que permite dar aos computadores a habilidade de aprender sem que sejam explicitamente programados para isso. Ela permite que computadores tomem decisões e interpretem dados de maneira automática, a partir de algoritmos. Temos vários tipos de aprendizagem, são elas: Supervisionada, não supervisionada, semi supervisionada, aprendizagem por reforço e deep learning.
	
Os algoritmos de aprendizagem de máquina, aprendem a induzir uma função ou hipótese capaz de resolver um problema a partir de dados que representam instâncias do problema a ser resolvido.

Um algoritmo é uma sequência finita de ações e regras que visam a solucionar um problema. Cada um deles aciona um diferente tipo de operação ao entrar em contato com os dados que o computador recebe. O resultado de todas as operações é o que possibilita o aprendizado da máquina.

Dessa forma, as máquinas aperfeiçoam as tarefas executadas, por meio de processamento de dados como imagens e números. Por isso o machine learning depende do Big Data para ser efetivo. O Big Data, por sua vez pode ser entendido de maneira simplória como uma imensa quantidade de dados. Mas calma, ainda irei falar mais em detalhes sobre isso. Por ora, vamos entender como o Machine Learning e a Inteligência Artificial ajudam a benefeciar a área de Bank Marketing.

**Machine Learning e o Bank Marketing**<br/>
Abaixo irei listar alguns benefícios do Machine Learning (ML) aplicado na área de Bank Marketing:
* **Atendimento ao cliente orientado por IA**: Existem muitas maneiras de tornar o atendimento ao cliente realmente orientado por IA ou, melhor dizer, orientado por dados. Por exemplo, com a ajuda da análise de dados, a instituição bancária pode descobrir as intenções de compra do cliente e oferecer um empréstimo flexível. Além disso, os principais bancos criam chatbots inteligentes que ajudam os clientes a interagir melhor com as empresas financeiras. Com a ajuda de aplicativos inteligentes, os clientes podem acompanhar automaticamente seus gastos, planejar seu orçamento e obter sugestões precisas de economia e investimento.
* **Segmentação de clientes**: Com ML, é possível encontrar características semelhantes e padrões entre os dados dos clientes. Dessa forma, o algoritmo de ML consegue separar os clientes em grupos, possibilitando que a equipe de Marketing possa direcionar os esforços de maneira individual para cada grupo de clientes.
* **Otimização de lances em anúncios**: os anúncios em buscadores funcionam no sistema de leilões de pesquisa. Ou seja, quem der o maior lance aparecerá em primeiro lugar nos resultados de pesquisa para uma determinada palavra-chave. Para fazer o lance perfeito, o marketing se utiliza do machine learning. Ele analisa milhões de dados para ajustar os lances em tempo real.
* **Prever os possíveis clientes**: Com base nos dados históricos da empresa, podemos coletar e entender qual é o perfil dos clientes. E com base nisso, prever a probabilidade do indivíduo adquirir determinado serviço. Como por exemplo, contrair um empréstimo, adquirir investimentos, dentre outros.

**Objetivo do Projeto**<br/>
Como objetivo específico do problema de negócio, irei aplicar técnicas de Machine Learning para identificar e prever ser se o cliente vai ou não adquirir CDBs (Certificado de Depósito Bancário). Este é um tipo de investimento muito comum e bastante conservador. De maneira rápida e simples, basicamente o cliente empresta o dinheiro para o banco e após de um prazo pré-estabelecido, ele resgata o dinheiro com o acréscimo de juros.

Como objetivo de estudo de tecnologia, estarei utilizando do início ao fim do projeto o framework Apache Spark, mais especificamente, o PySpark. PySpark é uma API Python para Apache SPARK que é denominado como o mecanismo de processamento analítico para aplicações de processamento de dados distribuídos em larga escala e aprendizado de máquina em tempo real, ou seja, para grandes volumes de dados, conhecido como Big Data.

**Sobre o Dataset**<br/>
Este conjunto de dados contém 20 atributos e 41189 registos relevantes para uma campanha de marketing direto de uma instituição bancária portuguesa. A campanha de marketing foi executada por meio de ligações telefônicas. O objetivo da classificação é prever se o cliente irá aderir (yes/no) ao CDB (variável y).<br/>

O dataset pode ser obtido clicando [aqui](https://www.kaggle.com/datasets/ruthgn/bank-marketing-data-set)

### Introdução ao Apache Spark

Apache Spark é uma estrutura de código aberto que simplifica o desenvolvimento e a eficiência dos trabalhos de análise de dados. Ele oferece suporte a uma ampla variedade de opções de API e linguagem com mais de 80 operadores de transformação e ação de dados que ocultam a complexidade da computação em cluster.

Com velocidades relatadas 100 vezes mais rápidas do que mecanismos de análise semelhantes, o Spark pode acessar fontes de dados variáveis e ser executado em várias plataformas, incluindo Hadoop, Apache Mesos, Kubernetes, de forma independente ou na nuvem. Seja processando dados em lote ou streaming, você verá um desempenho de alto nível devido ao agendador Spark DAG de última geração, um otimizador de consulta e um mecanismo de execução física.

> Caso você queira saber mais sobre o Apache Spark, eu recomendo fortemente a leitura do artigo "Spark: entenda sua função e saiba mais sobre essa ferramenta", publicado pelo blog XP Educação, que pode ser acessado clicando [aqui](https://blog.xpeducacao.com.br/apache-spark/)

### PySpark

PySpark é a colaboração do Apache Spark e do Python.

O Apache Spark é uma estrutura de computação em cluster de código aberto, construída em torno da velocidade, facilidade de uso e análise de streaming, enquanto o Python é uma linguagem de programação de alto nível e de uso geral. Ele fornece uma ampla variedade de bibliotecas e é usado principalmente para Machine Learning e Real-Time Streaming Analytics.

Em outras palavras, é uma API Python para Spark que permite aproveitar a simplicidade do Python e o poder do Apache Spark para domar o Big Data. 


**Links de Referência**:

* https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html
* https://sparkbyexamples.com/pyspark/pyspark-structtype-and-structfield/
* https://sparkbyexamples.com/pyspark/pyspark-map-transformation/

---

* https://spark.apache.org/docs/latest/ml-pipeline.html
* https://github.com/Krupique/Explorating-Data/blob/main/PreProcessamento-Medium/PreProcessamento.ipynb
* https://spark.apache.org/docs/latest/sql-data-sources-load-save-functions.html

* https://www.databricks.com/wp-content/uploads/hubfs/notebooks/spark2.0/ML%20persistence%20in%202.0.html

### Introdução

### Data Load and Packages Imports

In [141]:
from pyspark.sql import Row #Converte RDDs em objetos do tipo Row
from pyspark.ml.feature import StringIndexer #Converte strings em valores numéricos
from pyspark.ml.linalg import Vectors #Serve para criar um vetor denso
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [142]:
import sys
print(f'System Version: {sys.version}')
print(f'Spark Context Version: {sc.version}')

System Version: 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Spark Context Version: 3.0.3


In [143]:
# Spark Session - usada quando se trabalha com Dataframes no Spark
spSession = SparkSession.builder.master("local").appName("DSA-SparkMLLib").config("spark.some.config.option", "session").getOrCreate()

In [144]:
#rdd = sc.textFile('data/bank-marketing-dataset.csv')
rdd = sc.textFile('data/test.csv')

### Overview

In [145]:
type(rdd)

pyspark.rdd.RDD

In [146]:
rdd.count()

41189

In [147]:
# Listando os 5 primeiros registros
rdd.take(5)

['age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y',
 '56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no',
 '57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no',
 '37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no',
 '40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no']

In [148]:
header = rdd.first()
rdd_body = rdd.filter(lambda x: header not in x).map(lambda l: l.split(','))

list_columns = header.replace('.', '_').upper().split(',')
list_columns

['AGE',
 'JOB',
 'MARITAL',
 'EDUCATION',
 'DEFAULT',
 'HOUSING',
 'LOAN',
 'CONTACT',
 'MONTH',
 'DAY_OF_WEEK',
 'CAMPAIGN',
 'PDAYS',
 'PREVIOUS',
 'POUTCOME',
 'EMP_VAR_RATE',
 'CONS_PRICE_IDX',
 'CONS_CONF_IDX',
 'EURIBOR3M',
 'NR_EMPLOYED',
 'Y']

In [149]:
rdd_row = rdd_body.map(lambda p: Row(
    AGE = p[0], 
    JOB = p[1], 
    MARITAL = p[2],
    EDUCATION = p[3],
    DEFAULT = p[4],
    HOUSING = p[5],
    LOAN = p[6],
    CONTACT = p[7],
    MONTH = p[8],
    DAY_OF_WEEK = p[9],
    CAMPAIGN = p[10],
    PDAYS = p[11],
    PREVIOUS = p[12],
    POUTCOME = p[13],
    EMP_VAR_RATE = p[14],
    CONS_PRICE_IDX = p[15],
    CONS_CONF_IDX = p[16],
    EURIBOR3M = p[17],
    EMPLOYED = p[18],
    TARGET = p[19]
))

In [150]:
# Criando um Dataframe
rdd_df = spSession.createDataFrame(rdd_row)
rdd_df.cache()

DataFrame[AGE: string, JOB: string, MARITAL: string, EDUCATION: string, DEFAULT: string, HOUSING: string, LOAN: string, CONTACT: string, MONTH: string, DAY_OF_WEEK: string, CAMPAIGN: string, PDAYS: string, PREVIOUS: string, POUTCOME: string, EMP_VAR_RATE: string, CONS_PRICE_IDX: string, CONS_CONF_IDX: string, EURIBOR3M: string, EMPLOYED: string, TARGET: string]

In [151]:
# Find count for empty, None, Null, Nan with string literals.
from pyspark.sql.functions import col,isnan,when,count

rdd_na = rdd_df.select([count(when(col(c).contains('None') | col(c).contains('NULL') | \
                            (col(c) == '' ) | col(c).isNull() | isnan(c), c )).alias(c)
                    for c in rdd_df.columns])
rdd_na.show()

+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+-----+--------+--------+------------+--------------+-------------+---------+--------+------+
|AGE|JOB|MARITAL|EDUCATION|DEFAULT|HOUSING|LOAN|CONTACT|MONTH|DAY_OF_WEEK|CAMPAIGN|PDAYS|PREVIOUS|POUTCOME|EMP_VAR_RATE|CONS_PRICE_IDX|CONS_CONF_IDX|EURIBOR3M|EMPLOYED|TARGET|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+-----+--------+--------+------------+--------------+-------------+---------+--------+------+
|  0|  0|      1|        0|      1|      1|   0|      0|    0|          1|       0|    0|       0|       2|           0|             1|            0|        0|       0|     0|
+---+---+-------+---------+-------+-------+----+-------+-----+-----------+--------+-----+--------+--------+------------+--------------+-------------+---------+--------+------+



In [152]:
list_columns = rdd_df.columns

for column in list_columns:
    count = rdd_df.select(column).distinct().count()
    print(f'Column: {column}\tCount: {count}')

Column: AGE	Count: 78
Column: JOB	Count: 12
Column: MARITAL	Count: 5
Column: EDUCATION	Count: 8
Column: DEFAULT	Count: 4
Column: HOUSING	Count: 4
Column: LOAN	Count: 3
Column: CONTACT	Count: 2
Column: MONTH	Count: 10
Column: DAY_OF_WEEK	Count: 6
Column: CAMPAIGN	Count: 42
Column: PDAYS	Count: 27
Column: PREVIOUS	Count: 8
Column: POUTCOME	Count: 4
Column: EMP_VAR_RATE	Count: 10
Column: CONS_PRICE_IDX	Count: 27
Column: CONS_CONF_IDX	Count: 26
Column: EURIBOR3M	Count: 316
Column: EMPLOYED	Count: 11
Column: TARGET	Count: 2


### Handling Data Missing

Columns with missing values:
* MARITAL
* DEFAULT
* HOUSING
* DAY_OF_WEEK
* POUTCOME
* CONS_PRICE_IDX

In [153]:
def getDfGroup(rdd_df, column):
    df_group = spSession.createDataFrame(rdd_df.groupBy(['TARGET', column]).agg({column: 'count'}).collect())

    df_group = df_group.orderBy(['TARGET', column, f'count({column})'], ascending=[0, 1, 0])

    return df_group
    

**MARITAL**

In [154]:
df_group = getDfGroup(rdd_df, 'MARITAL')

df_group.collect()

[Row(TARGET='yes', MARITAL='divorced', count(MARITAL)=476),
 Row(TARGET='yes', MARITAL='married', count(MARITAL)=2532),
 Row(TARGET='yes', MARITAL='single', count(MARITAL)=1620),
 Row(TARGET='yes', MARITAL='unknown', count(MARITAL)=12),
 Row(TARGET='no', MARITAL='', count(MARITAL)=1),
 Row(TARGET='no', MARITAL='divorced', count(MARITAL)=4136),
 Row(TARGET='no', MARITAL='married', count(MARITAL)=22396),
 Row(TARGET='no', MARITAL='single', count(MARITAL)=9947),
 Row(TARGET='no', MARITAL='unknown', count(MARITAL)=68)]

A moda do valor nulo agrupada por target é `married`, contendo 22396 registros, portanto, é com esse valor que irei preencher.

**DEFAULT**

In [155]:
df_group = getDfGroup(rdd_df, 'DEFAULT')

df_group.collect()

[Row(TARGET='yes', DEFAULT='no', count(DEFAULT)=4197),
 Row(TARGET='yes', DEFAULT='unknown', count(DEFAULT)=443),
 Row(TARGET='no', DEFAULT='', count(DEFAULT)=1),
 Row(TARGET='no', DEFAULT='no', count(DEFAULT)=28391),
 Row(TARGET='no', DEFAULT='unknown', count(DEFAULT)=8153),
 Row(TARGET='no', DEFAULT='yes', count(DEFAULT)=3)]

A moda do atributo no valor nulo agrupado por target é `no`.

**EDUCATION**

In [156]:
df_group = getDfGroup(rdd_df, 'EDUCATION')

df_group.collect()

[Row(TARGET='yes', EDUCATION='basic.4y', count(EDUCATION)=428),
 Row(TARGET='yes', EDUCATION='basic.6y', count(EDUCATION)=188),
 Row(TARGET='yes', EDUCATION='basic.9y', count(EDUCATION)=473),
 Row(TARGET='yes', EDUCATION='high.school', count(EDUCATION)=1031),
 Row(TARGET='yes', EDUCATION='illiterate', count(EDUCATION)=4),
 Row(TARGET='yes', EDUCATION='professional.course', count(EDUCATION)=595),
 Row(TARGET='yes', EDUCATION='university.degree', count(EDUCATION)=1670),
 Row(TARGET='yes', EDUCATION='unknown', count(EDUCATION)=251),
 Row(TARGET='no', EDUCATION='basic.4y', count(EDUCATION)=3748),
 Row(TARGET='no', EDUCATION='basic.6y', count(EDUCATION)=2104),
 Row(TARGET='no', EDUCATION='basic.9y', count(EDUCATION)=5572),
 Row(TARGET='no', EDUCATION='high.school', count(EDUCATION)=8484),
 Row(TARGET='no', EDUCATION='illiterate', count(EDUCATION)=14),
 Row(TARGET='no', EDUCATION='professional.course', count(EDUCATION)=4648),
 Row(TARGET='no', EDUCATION='university.degree', count(EDUCATION)=

Neste atributo, temos valores `unknown`, porém, não irei considerar como valor nulo.

**HOUSING**

In [157]:
df_group = getDfGroup(rdd_df, 'HOUSING')

df_group.collect()

[Row(TARGET='yes', HOUSING='no', count(HOUSING)=2026),
 Row(TARGET='yes', HOUSING='unknown', count(HOUSING)=107),
 Row(TARGET='yes', HOUSING='yes', count(HOUSING)=2507),
 Row(TARGET='no', HOUSING='', count(HOUSING)=1),
 Row(TARGET='no', HOUSING='no', count(HOUSING)=16596),
 Row(TARGET='no', HOUSING='unknown', count(HOUSING)=883),
 Row(TARGET='no', HOUSING='yes', count(HOUSING)=19068)]

A moda do atributo no valor nulo agrupado por target é `yes`.

**DAY_OF_WEEK**

In [158]:
df_group = getDfGroup(rdd_df, 'DAY_OF_WEEK')

df_group.collect()

[Row(TARGET='yes', DAY_OF_WEEK='fri', count(DAY_OF_WEEK)=846),
 Row(TARGET='yes', DAY_OF_WEEK='mon', count(DAY_OF_WEEK)=847),
 Row(TARGET='yes', DAY_OF_WEEK='thu', count(DAY_OF_WEEK)=1045),
 Row(TARGET='yes', DAY_OF_WEEK='tue', count(DAY_OF_WEEK)=953),
 Row(TARGET='yes', DAY_OF_WEEK='wed', count(DAY_OF_WEEK)=949),
 Row(TARGET='no', DAY_OF_WEEK='', count(DAY_OF_WEEK)=1),
 Row(TARGET='no', DAY_OF_WEEK='fri', count(DAY_OF_WEEK)=6981),
 Row(TARGET='no', DAY_OF_WEEK='mon', count(DAY_OF_WEEK)=7666),
 Row(TARGET='no', DAY_OF_WEEK='thu', count(DAY_OF_WEEK)=7578),
 Row(TARGET='no', DAY_OF_WEEK='tue', count(DAY_OF_WEEK)=7137),
 Row(TARGET='no', DAY_OF_WEEK='wed', count(DAY_OF_WEEK)=7185)]

A moda do atributo no valor nulo agrupado por target é `mon`.

**POUTCOME**

In [159]:
df_group = getDfGroup(rdd_df, 'POUTCOME')

df_group.collect()

[Row(TARGET='yes', POUTCOME='failure', count(POUTCOME)=605),
 Row(TARGET='yes', POUTCOME='nonexistent', count(POUTCOME)=3141),
 Row(TARGET='yes', POUTCOME='success', count(POUTCOME)=894),
 Row(TARGET='no', POUTCOME='', count(POUTCOME)=2),
 Row(TARGET='no', POUTCOME='failure', count(POUTCOME)=3647),
 Row(TARGET='no', POUTCOME='nonexistent', count(POUTCOME)=32420),
 Row(TARGET='no', POUTCOME='success', count(POUTCOME)=479)]

A moda do atributo no valor nulo agrupado por target é `nonexistent`.

**CONS_PRICE_IDX**

In [160]:
df_group = getDfGroup(rdd_df, 'CONS_PRICE_IDX')

df_group.collect()

[Row(TARGET='yes', CONS_PRICE_IDX='92.20100000000001', count(CONS_PRICE_IDX)=264),
 Row(TARGET='yes', CONS_PRICE_IDX='92.37899999999999', count(CONS_PRICE_IDX)=106),
 Row(TARGET='yes', CONS_PRICE_IDX='92.431', count(CONS_PRICE_IDX)=180),
 Row(TARGET='yes', CONS_PRICE_IDX='92.469', count(CONS_PRICE_IDX)=66),
 Row(TARGET='yes', CONS_PRICE_IDX='92.649', count(CONS_PRICE_IDX)=168),
 Row(TARGET='yes', CONS_PRICE_IDX='92.713', count(CONS_PRICE_IDX)=88),
 Row(TARGET='yes', CONS_PRICE_IDX='92.756', count(CONS_PRICE_IDX)=1),
 Row(TARGET='yes', CONS_PRICE_IDX='92.84299999999999', count(CONS_PRICE_IDX)=126),
 Row(TARGET='yes', CONS_PRICE_IDX='92.89299999999999', count(CONS_PRICE_IDX)=524),
 Row(TARGET='yes', CONS_PRICE_IDX='92.963', count(CONS_PRICE_IDX)=264),
 Row(TARGET='yes', CONS_PRICE_IDX='93.075', count(CONS_PRICE_IDX)=442),
 Row(TARGET='yes', CONS_PRICE_IDX='93.2', count(CONS_PRICE_IDX)=190),
 Row(TARGET='yes', CONS_PRICE_IDX='93.369', count(CONS_PRICE_IDX)=150),
 Row(TARGET='yes', CONS_PR

A moda do atributo no valor nulo agrupado por target é `93.91799999999999`.

In [161]:
def verificarNA(c):
    c = c.upper()
    if c == 'NONE' or c == 'NULL' or c == '' or c == 'NAN':
        return True
    return False

def mapNA(x):
    AGE = x.AGE
    JOB = x.JOB
    MARITAL = x.MARITAL
    EDUCATION = x.EDUCATION
    DEFAULT = x.DEFAULT
    HOUSING = x.HOUSING
    LOAN = x.LOAN
    CONTACT = x.CONTACT
    MONTH = x.MONTH
    DAY_OF_WEEK = x.DAY_OF_WEEK
    CAMPAIGN = x.CAMPAIGN
    PDAYS = x.PDAYS
    PREVIOUS = x.PREVIOUS
    POUTCOME = x.POUTCOME
    EMP_VAR_RATE = x.EMP_VAR_RATE
    CONS_PRICE_IDX = x.CONS_PRICE_IDX
    CONS_CONF_IDX = x.CONS_CONF_IDX
    EURIBOR3M = x.EURIBOR3M
    EMPLOYED = x.EMPLOYED
    TARGET = x.TARGET
    
    #Corrigindo valores missing
    if verificarNA(x.MARITAL):
        MARITAL = 'married'
        
    if verificarNA(x.DEFAULT):
        DEFAULT = 'no'
        
    if verificarNA(x.HOUSING):
        HOUSING = 'yes'
        
    if verificarNA(x.DAY_OF_WEEK):
        DAY_OF_WEEK = 'mon'
        
    if verificarNA(x.POUTCOME):
        POUTCOME = 'nonexistent'
        
    if verificarNA(x.CONS_PRICE_IDX):
        CONS_PRICE_IDX = '93.91799999999999'
    
    
    return (AGE, JOB, MARITAL, EDUCATION, DEFAULT, HOUSING, LOAN, CONTACT, MONTH, DAY_OF_WEEK, CAMPAIGN, PDAYS, PREVIOUS, POUTCOME, EMP_VAR_RATE, CONS_PRICE_IDX, CONS_CONF_IDX, EURIBOR3M, EMPLOYED, TARGET)



In [162]:
rdd_row = rdd_df.rdd.map(lambda x: mapNA(x))

# Criando um Dataframe
rdd_df = spSession.createDataFrame(rdd_row, schema=list_columns)
rdd_df.cache()

DataFrame[AGE: string, JOB: string, MARITAL: string, EDUCATION: string, DEFAULT: string, HOUSING: string, LOAN: string, CONTACT: string, MONTH: string, DAY_OF_WEEK: string, CAMPAIGN: string, PDAYS: string, PREVIOUS: string, POUTCOME: string, EMP_VAR_RATE: string, CONS_PRICE_IDX: string, CONS_CONF_IDX: string, EURIBOR3M: string, EMPLOYED: string, TARGET: string]

In [163]:
rdd_df.show(5)

+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+-----+--------+-----------+------------+--------------+-------------+---------+--------+------+
|AGE|      JOB|MARITAL|  EDUCATION|DEFAULT|HOUSING|LOAN|  CONTACT|MONTH|DAY_OF_WEEK|CAMPAIGN|PDAYS|PREVIOUS|   POUTCOME|EMP_VAR_RATE|CONS_PRICE_IDX|CONS_CONF_IDX|EURIBOR3M|EMPLOYED|TARGET|
+---+---------+-------+-----------+-------+-------+----+---------+-----+-----------+--------+-----+--------+-----------+------------+--------------+-------------+---------+--------+------+
| 56|housemaid|married|   basic.4y|     no|     no|  no|telephone|  may|        mon|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|  5191.0|    no|
| 57| services|married|high.school|unknown|     no|  no|telephone|  may|        mon|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|  5191.0|    no|
| 37| services|married|high.school|     no|    yes|  no

### Alterando o tipo de dado

In [164]:
for column in list_columns:
    print(f'{column}: Distinct count: {rdd_df.select(column).distinct().count()}')
    rdd_df.select(column).show(5)
    rdd_df.select(column).distinct()

AGE: Distinct count: 78
+---+
|AGE|
+---+
| 56|
| 57|
| 37|
| 40|
| 56|
+---+
only showing top 5 rows

JOB: Distinct count: 12
+---------+
|      JOB|
+---------+
|housemaid|
| services|
| services|
|   admin.|
| services|
+---------+
only showing top 5 rows

MARITAL: Distinct count: 4
+-------+
|MARITAL|
+-------+
|married|
|married|
|married|
|married|
|married|
+-------+
only showing top 5 rows

EDUCATION: Distinct count: 8
+-----------+
|  EDUCATION|
+-----------+
|   basic.4y|
|high.school|
|high.school|
|   basic.6y|
|high.school|
+-----------+
only showing top 5 rows

DEFAULT: Distinct count: 3
+-------+
|DEFAULT|
+-------+
|     no|
|unknown|
|     no|
|     no|
|     no|
+-------+
only showing top 5 rows

HOUSING: Distinct count: 3
+-------+
|HOUSING|
+-------+
|     no|
|     no|
|    yes|
|     no|
|     no|
+-------+
only showing top 5 rows

LOAN: Distinct count: 3
+----+
|LOAN|
+----+
|  no|
|  no|
|  no|
|  no|
| yes|
+----+
only showing top 5 rows

CONTACT: Distinct coun

---


In [165]:
from pyspark.sql.types import IntegerType, BooleanType, DateType, StringType, FloatType

rdd_df = rdd_df.withColumn('AGE', col('AGE').cast(IntegerType()))
rdd_df = rdd_df.withColumn('CAMPAIGN ', col('CAMPAIGN').cast(IntegerType()))
rdd_df = rdd_df.withColumn('AGE', col('AGE').cast(IntegerType()))
rdd_df = rdd_df.withColumn('PDAYS', col('PDAYS').cast(IntegerType()))
rdd_df = rdd_df.withColumn('PREVIOUS', col('PREVIOUS').cast(IntegerType()))

rdd_df = rdd_df.withColumn('EMP_VAR_RATE', col('EMP_VAR_RATE').cast(FloatType()))
rdd_df = rdd_df.withColumn('CONS_PRICE_IDX', col('CONS_PRICE_IDX').cast(FloatType()))
rdd_df = rdd_df.withColumn('CONS_CONF_IDX', col('CONS_CONF_IDX').cast(FloatType()))
rdd_df = rdd_df.withColumn('EURIBOR3M', col('EURIBOR3M').cast(FloatType()))
rdd_df = rdd_df.withColumn('EMPLOYED', col('EMPLOYED').cast(FloatType()))

In [166]:
rdd_df.dtypes

[('AGE', 'int'),
 ('JOB', 'string'),
 ('MARITAL', 'string'),
 ('EDUCATION', 'string'),
 ('DEFAULT', 'string'),
 ('HOUSING', 'string'),
 ('LOAN', 'string'),
 ('CONTACT', 'string'),
 ('MONTH', 'string'),
 ('DAY_OF_WEEK', 'string'),
 ('CAMPAIGN', 'string'),
 ('PDAYS', 'int'),
 ('PREVIOUS', 'int'),
 ('POUTCOME', 'string'),
 ('EMP_VAR_RATE', 'float'),
 ('CONS_PRICE_IDX', 'float'),
 ('CONS_CONF_IDX', 'float'),
 ('EURIBOR3M', 'float'),
 ('EMPLOYED', 'float'),
 ('TARGET', 'string'),
 ('CAMPAIGN ', 'int')]

### Normalização e Encoding dos dados

In [167]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.functions import vector_to_array
import pyspark.sql.functions as F

In [168]:
columns_str = ['JOB', 'MARITAL', 'EDUCATION', 'DEFAULT', 'HOUSING', 
               'LOAN', 'CONTACT', 'MONTH', 'DAY_OF_WEEK', 'POUTCOME', 'TARGET']

In [169]:
# Função expandir as colunas que foram geradas pelo OneHotEncoding.
def _oneHotEncoder(df, column):
    
    alias = f'_{column}'
    df_col_onehot = df.select('*', vector_to_array(column).alias(alias))


    num_categories = len(df_col_onehot.first()[alias])
    cols_expanded = [(F.col(alias)[i]) for i in range(num_categories)]
    df_cols_onehot = df_col_onehot.select('*', *cols_expanded)
    
    return df_cols_onehot, cols_expanded


In [170]:
# LabelEncoder
indexers = [ StringIndexer(inputCol=c, outputCol="OHE_{0}".format(c)) for c in columns_str]

# Criação do array de Encoders
encoders = [
    OneHotEncoder(
        inputCol=indexer.getOutputCol(),
        outputCol="_{0}".format(indexer.getOutputCol())) 
    for indexer in indexers
]

In [171]:
# Aplica os dois métodos: LabelEncoder e OneHotEncoder
pipeline = Pipeline(stages=indexers + encoders)
df_temp = pipeline.fit(rdd_df).transform(rdd_df)

# Obtendo todas as colunas de saída do Encoder
encoder_columns = [encoder.getOutputCol() for encoder in encoders]

# Expandindo o array gerado pelo OneHotEncoding para Colunas individuais
ohe_columns = []
for column in encoder_columns: 
    df_temp, _ohe = _oneHotEncoder(df_temp, column)
    
    ohe_columns.extend(_ohe)

In [172]:
df_res = df_temp.select('AGE', 'PDAYS', 'PREVIOUS', 
                         'EMP_VAR_RATE', 'CONS_PRICE_IDX', 
                         'CONS_CONF_IDX', 'EURIBOR3M', 'EMPLOYED', 
                         'TARGET', 'CAMPAIGN', *ohe_columns)

df_res.show(1)

+---+-----+--------+------------+--------------+-------------+---------+--------+------+--------+------------+------------+------------+------------+------------+------------+------------+------------+------------+------------+-------------+----------------+----------------+----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+----------------+----------------+----------------+----------------+-------------+-------------+----------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+---------------+
|AGE|PDAYS|PREVIOUS|EMP_VAR_RATE|CONS_PRICE_IDX|CONS_CONF_IDX|EURIBOR3M|EMPLOYED|TARGET|CAMPAIGN|__OHE_JOB[0]|__OHE_JOB[1]|__OHE_JOB[2]|__OHE_JOB[3]|__OHE_JOB[4]|__OHE_JOB[5]|__OHE_JOB[6]|__

### Pré Processamento dos Dados

In [173]:
def transformaVar(row) :
    obj = (row['TARGET'], row['__OHE_TARGET[0]'], Vectors.dense([
                                                            row['AGE'],
                                                            row['PDAYS'],
                                                            row['PREVIOUS'],
                                                            row['EMP_VAR_RATE'],
                                                            row['CONS_PRICE_IDX'],
                                                            row['CONS_CONF_IDX'],
                                                            row['EURIBOR3M'],
                                                            row['EMPLOYED'],
                                                            row['CAMPAIGN'],
                                                            row['__OHE_JOB[0]'],
                                                            row['__OHE_JOB[1]'],
                                                            row['__OHE_JOB[2]'],
                                                            row['__OHE_JOB[3]'],
                                                            row['__OHE_JOB[4]'],
                                                            row['__OHE_JOB[5]'],
                                                            row['__OHE_JOB[6]'],
                                                            row['__OHE_JOB[7]'],
                                                            row['__OHE_JOB[8]'],
                                                            row['__OHE_JOB[9]'],
                                                            row['__OHE_JOB[10]'],
                                                            row['__OHE_MARITAL[0]'],
                                                            row['__OHE_MARITAL[1]'],
                                                            row['__OHE_MARITAL[2]'],
                                                            row['__OHE_EDUCATION[0]'],
                                                            row['__OHE_EDUCATION[1]'],
                                                            row['__OHE_EDUCATION[2]'],
                                                            row['__OHE_EDUCATION[3]'],
                                                            row['__OHE_EDUCATION[4]'],
                                                            row['__OHE_EDUCATION[5]'],
                                                            row['__OHE_EDUCATION[6]'],
                                                            row['__OHE_DEFAULT[0]'],
                                                            row['__OHE_DEFAULT[1]'],
                                                            row['__OHE_HOUSING[0]'],
                                                            row['__OHE_HOUSING[1]'],
                                                            row['__OHE_LOAN[0]'],
                                                            row['__OHE_LOAN[1]'],
                                                            row['__OHE_CONTACT[0]'],
                                                            row['__OHE_MONTH[0]'],
                                                            row['__OHE_MONTH[1]'],
                                                            row['__OHE_MONTH[2]'],
                                                            row['__OHE_MONTH[3]'],
                                                            row['__OHE_MONTH[4]'],
                                                            row['__OHE_MONTH[5]'],
                                                            row['__OHE_MONTH[6]'],
                                                            row['__OHE_MONTH[7]'],
                                                            row['__OHE_MONTH[8]'],
                                                            row['__OHE_DAY_OF_WEEK[0]'],
                                                            row['__OHE_DAY_OF_WEEK[1]'],
                                                            row['__OHE_DAY_OF_WEEK[2]'],
                                                            row['__OHE_DAY_OF_WEEK[3]'],
                                                            row['__OHE_POUTCOME[0]'],
                                                            row['__OHE_POUTCOME[1]']
                                                            ]))
    return obj

In [174]:
rdd_processing = df_res.rdd.map(transformaVar)

rdd_processing.take(1)

[('no',
  1.0,
  DenseVector([56.0, 999.0, 0.0, 1.1, 93.994, -36.4, 4.857, 5191.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0]))]

In [175]:
df_processing = spSession.createDataFrame(rdd_processing, ["target", "label", "features"])
df_processing.show(10)
df_processing.cache()

+------+-----+--------------------+
|target|label|            features|
+------+-----+--------------------+
|    no|  1.0|[56.0,999.0,0.0,1...|
|    no|  1.0|[57.0,999.0,0.0,1...|
|    no|  1.0|[37.0,999.0,0.0,1...|
|    no|  1.0|[40.0,999.0,0.0,1...|
|    no|  1.0|[56.0,999.0,0.0,1...|
|    no|  1.0|[45.0,999.0,0.0,1...|
|    no|  1.0|[59.0,999.0,0.0,1...|
|    no|  1.0|[41.0,999.0,0.0,1...|
|    no|  1.0|[24.0,999.0,0.0,1...|
|    no|  1.0|[25.0,999.0,0.0,1...|
+------+-----+--------------------+
only showing top 10 rows



DataFrame[target: string, label: double, features: vector]

### Escala dos Dados

In [176]:
from pyspark.ml.feature import RobustScaler, StandardScaler, MinMaxScaler, Normalizer

In [177]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')

### Divisão em treino e teste

In [257]:
# Dados de Treino e de Teste
(dados_treino, dados_teste, dados_valid) = df_processing.randomSplit([0.78, 0.2, 0.02])

dados_treino.count(), dados_teste.count(), dados_valid.count()

(32007, 8381, 800)

#### Tratamento das classes desbalanceadas

In [258]:
dados_treino.groupBy('target').count().show()

+------+-----+
|target|count|
+------+-----+
|    no|28379|
|   yes| 3628|
+------+-----+



In [259]:
df_target_1 = dados_treino[dados_treino['target'] == 'no']
df_target_0 = dados_treino[dados_treino['target'] == 'yes']

df_target_1.count(), df_target_0.count()

(28379, 3628)

In [260]:
from math import floor

In [261]:
fraction = float(floor(df_target_1.count() / df_target_0.count()))

df_target_0_sample = df_target_0.sample(True, fraction, 1234)

In [262]:
diff = df_target_0_sample.count() - df_target_0.count()

df_temp = spSession.createDataFrame(df_target_0_sample.collect()[0:diff])

df_temp = df_temp.unionAll(df_target_0)

In [263]:
dados_treino =  df_temp.unionAll(df_target_1)

In [264]:
dados_treino.count()

53908

In [265]:
dados_treino.groupBy('target').count().show()

+------+-----+
|target|count|
+------+-----+
|    no|28379|
|   yes|25529|
+------+-----+



### Machine Learning

In [266]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import LinearSVC

In [267]:
#dtClassifer = RandomForestClassifier(labelCol = "label", featuresCol = "features")
#dtClassifer = GBTClassifier(labelCol = "label", featuresCol = "features_scaled")
dtClassifer = LinearSVC(labelCol = "label", featuresCol = "features")

### Aplicando Pipeline, Treinando e Prevendo o Modelo

In [268]:
pipeline = Pipeline(stages=[scaler, dtClassifer])

model = pipeline.fit(dados_treino)

In [269]:
# Previsões com dados de teste
previsoes = model.transform(dados_teste)

#previsoes.select("prediction","target","label").collect()

### Avaliação do Modelo

In [270]:
# Avaliando a acurácia
avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "f1")
avaliador.evaluate(previsoes) 

0.7660945495380374

In [271]:
# Resumindo as previsões - Confusion Matrix
previsoes.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 5336|
|  0.0|       1.0|  267|
|  1.0|       0.0| 2123|
|  0.0|       0.0|  655|
+-----+----------+-----+



### Salvando o Modelo

In [272]:
#Save model
model.write().overwrite().save('models/modelo_v1')

### Carregando o Modelo

In [273]:
from pyspark.ml import Pipeline, PipelineModel

In [274]:
model_load = PipelineModel.load('models/modelo_v1')

In [275]:
preds_valid = model_load.transform(dados_valid)

avaliador = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "f1")
avaliador.evaluate(preds_valid) 

0.7906905011188439

In [276]:
# Resumindo as previsões - Confusion Matrix
preds_valid.groupBy("label","prediction").count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  523|
|  0.0|       1.0|   16|
|  1.0|       0.0|  187|
|  0.0|       0.0|   74|
+-----+----------+-----+



### Considerações Finais